<a href="https://colab.research.google.com/github/RezuanChowdhuryRifat/Drug-Repurposing/blob/main/drug_repurposing_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SARS COV-2 Knowledge Graph

Load packages


In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import numpy as np
import pandas as pd
import time
import re
import math
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.modules import Module
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils import add_remaining_self_loops, add_self_loops
from torch_geometric.utils import to_undirected
from torch_geometric.nn import GCNConv, SAGEConv,GAE, VGAE

In [4]:
embedding_size=128

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Load Preprocessed files

In [2]:
!pip install "pandas<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 33.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [6]:
le=pickle.load(open('/content/drive/MyDrive/Drug Repurposing/LabelEncoder_v2.pkl', 'rb'))
edge_index=pickle.load(open('/content/drive/MyDrive/Drug Repurposing/edge_index_v2.pkl','rb'))
node_feature_np=pickle.load(open('/content/drive/MyDrive/Drug Repurposing/node_feature_v2.pkl','rb'))

In [7]:
node_feature=torch.tensor(node_feature_np, dtype=torch.float)

In [8]:
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)

In [9]:
edge_attr_dict={'gene-drug':0,'gene-gene':1,'bait-gene':2, 'gene-phenotype':3, 'drug-phenotype':4}
edge_index['type']=edge_index['type'].apply(lambda x: edge_attr_dict[x])

In [10]:
edge_index['type'].value_counts()

0    419671
1      6240
3      2190
4      1700
2       247
Name: type, dtype: int64

In [11]:
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)

In [12]:
data = Data(x=node_feature,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr
           )

In [13]:
data.num_features, data.num_nodes,data.num_edges

(400, 14675, 430048)

In [14]:
edge_attr.size()

torch.Size([430048])

In [15]:
data.contains_isolated_nodes(), data.is_directed()

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)


(False, True)

In [16]:
device=torch.device(0)

**Batch**

In [17]:
def train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1):
    r"""Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.

    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)

    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.

    num_nodes = data.num_nodes
    row, col = data.edge_index
    #data.edge_index = None
    attr = data.edge_attr

    # Return upper triangular portion.
    #mask = row < col
    #row, col = row[mask], col[mask]

    n_v = int(math.floor(val_ratio * row.size(0)))
    n_t = int(math.floor(test_ratio * row.size(0)))

    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]
    attr=attr[perm]

    r, c = row[:n_v], col[:n_v]
    data.val_pos_edge_index = torch.stack([r, c], dim=0)
    data.val_pos_edge_attr = attr[:n_v]

    r, c = row[n_v:n_v + n_t], col[n_v:n_v + n_t]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)
    data.test_post_edge_attr = attr[n_v:n_v + n_t]

    r, c = row[n_v + n_t:], col[n_v + n_t:]
    data.train_pos_edge_index = torch.stack([r, c], dim=0)
    data.train_pos_edge_attr = attr[n_v+n_t:]

    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero().t()
    perm = random.sample(range(neg_row.size(0)),
                         min(n_v + n_t, neg_row.size(0)))
    perm = torch.tensor(perm)
    perm = perm.to(torch.long)
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    neg_adj_mask[neg_row, neg_col] = 0
    data.train_neg_adj_mask = neg_adj_mask

    row, col = neg_row[:n_v], neg_col[:n_v]
    data.val_neg_edge_index = torch.stack([row, col], dim=0)

    row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

In [18]:
print(device)

cuda:0


In [19]:
data_split=train_test_split_edges(data, test_ratio=0.1, val_ratio=0)
x,train_pos_edge_index,train_pos_edge_attr = data_split.x.to(device), data_split.train_pos_edge_index.to(device), data_split.train_pos_edge_attr.to(device)

In [20]:
train_pos_edge_index, train_pos_edge_attr=add_remaining_self_loops(train_pos_edge_index,train_pos_edge_attr)

In [21]:
pd.Series(train_pos_edge_attr.cpu().numpy()).value_counts()

0    377697
1     20242
3      1973
4      1538
2       220
dtype: int64

In [22]:
x,train_pos_edge_index,train_pos_edge_attr = Variable(x),Variable(train_pos_edge_index),Variable(train_pos_edge_attr)

# Learning Models

Define VGAE model

In [23]:
class Encoder_VGAE(nn.Module):
    def __init__(self, in_channels, out_channels, isClassificationTask=False):
        super(Encoder_VGAE, self).__init__()
        self.isClassificationTask=isClassificationTask
        self.conv_gene_drug=  SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_gene = SAGEConv(in_channels, 2*out_channels, )
        self.conv_bait_gene = SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_phenotype = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_phenotype = SAGEConv(in_channels, 2*out_channels)

        self.bn = nn.BatchNorm1d(5*2*out_channels)
        #variational encoder
        self.conv_mu = SAGEConv(5*2*out_channels, out_channels, )
        self.conv_logvar = SAGEConv(5*2*out_channels, out_channels,)

    def forward(self,x,edge_index,edge_attr):

        x = F.dropout(x, training=self.training)

        index_gene_drug=(edge_attr==0).nonzero().reshape(1,-1)[0]
        edge_index_gene_drug=edge_index[:, index_gene_drug]

        index_gene_gene=(edge_attr==1).nonzero().reshape(1,-1)[0]
        edge_index_gene_gene=edge_index[:, index_gene_gene]

        index_bait_gene=(edge_attr==2).nonzero().reshape(1,-1)[0]
        edge_index_bait_gene=edge_index[:, index_bait_gene]

        index_gene_phenotype=(edge_attr==3).nonzero().reshape(1,-1)[0]
        edge_index_gene_phenotype=edge_index[:, index_gene_phenotype]

        index_drug_phenotype=(edge_attr==4).nonzero().reshape(1,-1)[0]
        edge_index_drug_phenotype=edge_index[:, index_drug_phenotype]


        x_gene_drug = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_gene_drug)), p=0.5, training=self.training, )
        x_gene_gene = F.dropout(F.relu(self.conv_gene_gene(x,edge_index_gene_gene)), p=0.5, training=self.training)
        x_bait_gene = F.dropout(F.relu(self.conv_bait_gene(x,edge_index_bait_gene)), p=0.1, training=self.training)
        x_gene_phenotype = F.dropout(F.relu(self.conv_gene_phenotype(x,edge_index_gene_phenotype)), training=self.training)
        x_drug_phenotype = F.dropout(F.relu(self.conv_drug_phenotype(x,edge_index_drug_phenotype)), training=self.training)

        x=self.bn(torch.cat([x_gene_drug,x_gene_gene,x_bait_gene,x_gene_phenotype,x_drug_phenotype],dim=1))

        return self.conv_mu(x,edge_index), self.conv_logvar(x,edge_index)

In [24]:
model=VGAE(Encoder_VGAE(node_feature.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [25]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index, train_pos_edge_attr)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    print(loss.item())

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z=model.encode(x, train_pos_edge_index,train_pos_edge_attr)
    return model.test(z, pos_edge_index, neg_edge_index)

In [26]:
#DRKG's accuracy for comparison
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index )

(0.2965669438757359, 0.4280773528040785)

In [27]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

22.258056640625
Epoch: 001, AUC: 0.6249, AP: 0.5177
18.37226676940918
Epoch: 002, AUC: 0.7667, AP: 0.6993
15.807873725891113
Epoch: 003, AUC: 0.7267, AP: 0.6480
15.341449737548828
Epoch: 004, AUC: 0.6923, AP: 0.6173
16.169008255004883
Epoch: 005, AUC: 0.7188, AP: 0.6450
15.32749080657959
Epoch: 006, AUC: 0.7603, AP: 0.6967
14.353047370910645
Epoch: 007, AUC: 0.8016, AP: 0.7688
13.345233917236328
Epoch: 008, AUC: 0.8206, AP: 0.7901
12.078516006469727
Epoch: 009, AUC: 0.8326, AP: 0.7885


Node Embedding

In [28]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

In [29]:
data_path="/content/drive/MyDrive/Drug Repurposing/"

Save the new embedding

In [30]:
pickle.dump(z_np, open(data_path+'node_embedding_v2'+'.pkl', 'wb'))

Save the torach model

In [31]:
torch.save(model.state_dict(), data_path+'VAE_encoders_v2'+'.pkl')

In [32]:
model.load_state_dict(torch.load(data_path+'VAE_encoders_v2'+'.pkl'))
model.eval()

VGAE(
  (encoder): Encoder_VGAE(
    (conv_gene_drug): SAGEConv(400, 256, aggr=mean)
    (conv_gene_gene): SAGEConv(400, 256, aggr=mean)
    (conv_bait_gene): SAGEConv(400, 256, aggr=mean)
    (conv_gene_phenotype): SAGEConv(400, 256, aggr=mean)
    (conv_drug_phenotype): SAGEConv(400, 256, aggr=mean)
    (bn): BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_mu): SAGEConv(1280, 128, aggr=mean)
    (conv_logvar): SAGEConv(1280, 128, aggr=mean)
  )
  (decoder): InnerProductDecoder()
)

# Ranking Models

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score

In [34]:
topk=300
types=np.array([item.split('_')[0] for item in le.classes_ ])

Load drugs under clinical trial

In [35]:
#label
trials=pd.read_excel('/content/drive/MyDrive/Drug Repurposing/All_trails_5_24.xlsx',header=1,index_col=0)
trials_drug=set([drug.strip().upper() for lst in trials.loc[trials['study_category'].apply(lambda x: 'drug' in x.lower()),'intervention'].apply(lambda x: re.split(r'[+|/|,]',x.replace(' vs. ', '/').replace(' vs ', '/').replace(' or ', '/').replace(' with and without ', '/').replace(' /wo ', '/').replace(' /w ', '/').replace(' and ', '/').replace(' - ', '/').replace(' (', '/').replace(') ', '/'))).values for drug in lst])
drug_labels=[1 if drug.split('_')[1] in trials_drug else 0 for drug in le.classes_[types=='drug'] ]

**BPR NN Loss**

In [36]:
seed=70
indices = np.arange(len(drug_labels))
X_train, X_test, y_train, y_test,indices_train,indices_test=train_test_split(z_np[types=='drug'],drug_labels,indices, test_size=0.5,random_state=seed,)

In [37]:
#Variable wrapping for torch.tensor
_X_train, _y_train=Variable(torch.tensor(X_train,dtype=torch.float).to(device)), Variable(torch.tensor(y_train,dtype=torch.float).to(device))
_X_test, _y_test=Variable(torch.tensor(X_test,dtype=torch.float).to(device)), Variable(torch.tensor(y_test,dtype=torch.float).to(device))

In [38]:
class Classifier(nn.Module):
    def __init__(self,embedding_dim=embedding_size):
        super(Classifier, self).__init__()
        self.fc1=nn.Linear(embedding_dim,embedding_dim)
        self.fc2=nn.Linear(embedding_dim,1)
        self.bn=nn.BatchNorm1d(embedding_dim)
    def forward(self, x):
        residual1 = x
        x = F.dropout(x, training=self.training)
        x= self.bn(F.dropout(F.relu(self.fc1(x)),training=self.training))
        x += residual1
        return self.fc2(x)

In [39]:

from torch.utils.data import BatchSampler, WeightedRandomSampler
class BPRLoss(nn.Module):
    def __init__(self, num_neg_samples):
        super(BPRLoss, self).__init__()
        self.num_neg_samples=num_neg_samples

    def forward(self, output, label):
        positive_output=output[label==1]
        negative_output=output[label!=1]

        #negative sample proportional to the high values
        negative_sampler=WeightedRandomSampler(negative_output-min(negative_output), num_samples=self.num_neg_samples*len(positive_output),replacement=True)
        negative_sample_output=negative_output[torch.tensor(list(BatchSampler(negative_sampler, batch_size=len(positive_output),drop_last=True)),dtype=torch.long).t()]
        return -(positive_output.view(-1,1)-negative_sample_output).sigmoid().log().mean()

In [40]:
clf=Classifier(embedding_size).to(device)
optimizer=torch.optim.Adam(clf.parameters())
criterion=BPRLoss(num_neg_samples=15)

In [43]:
best_auprc=0
for epoch in range(30):
    clf.train()
    optimizer.zero_grad()
    out = clf(_X_train)
    loss=criterion(out.squeeze(), _y_train)
    loss.backward()
    optimizer.step()
    print('training loss',loss.item())

    clf.eval()
    print('test loss', criterion(clf(_X_test).squeeze(), _y_test).item())
    prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
    auprc=metrics.average_precision_score(y_test,prob)
    if auprc>best_auprc:
        best_auproc=auprc
        torch.save(clf, data_path+'nn_clf-temp_v2.pt')

training loss 0.42777496576309204
test loss 0.3844509422779083
training loss 0.44530823826789856
test loss 0.405448853969574
training loss 0.3897322118282318
test loss 0.3849104344844818
training loss 0.4157465994358063
test loss 0.39715251326560974
training loss 0.39845943450927734
test loss 0.3831035792827606
training loss 0.3549855947494507
test loss 0.36529678106307983
training loss 0.40289604663848877
test loss 0.3786458373069763
training loss 0.3604772686958313
test loss 0.37360137701034546
training loss 0.4170481860637665
test loss 0.3718845546245575
training loss 0.5033170580863953
test loss 0.3902912735939026
training loss 0.39711833000183105
test loss 0.3668919503688812
training loss 0.3956127166748047
test loss 0.3674808740615845
training loss 0.4476303458213806
test loss 0.37228211760520935
training loss 0.4190622866153717
test loss 0.36150938272476196
training loss 0.40032827854156494
test loss 0.37298762798309326
training loss 0.40773171186447144
test loss 0.3798282146453

In [44]:
clf.load_state_dict(torch.load('/content/drive/MyDrive/Drug Repurposing/nn_clf-temp_v2.pt').state_dict())

<All keys matched successfully>

In [45]:
#Compute AUC
clf.eval()

prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
print("AUROC", metrics.roc_auc_score(y_test,prob))
print("AUPRC", metrics.average_precision_score(y_test,prob))

AUROC 0.8753525098702764
AUPRC 0.2137688504624097


In [46]:
top_items_idx=np.argsort(-clf(torch.tensor(z_np[types=='drug'],dtype=torch.float).to(device)).squeeze().detach().cpu().numpy())

**Baseline Model**

In [47]:
clf=LogisticRegression().fit(X_train,y_train)
print("Logit AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("Logit AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

Logit AUROC 0.8701738194123981
Logit AUPRC 0.21033463554996845


In [48]:
clf=GradientBoostingClassifier().fit(X_train,y_train)
print("XGBoost AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("XGBoost AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

XGBoost AUROC 0.8595344306004205
XGBoost AUPRC 0.11860170708989237


In [49]:
clf=RandomForestClassifier().fit(X_train,y_train)
print("rf AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("rf AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

rf AUROC 0.8369225247397837
rf AUPRC 0.12084156255218911


In [50]:
clf=make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True)).fit(X_train,y_train)
print("svm AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("svm AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

svm AUROC 0.8029739014510587
svm AUPRC 0.16606332180221894


In [51]:
topk_drugs=pd.DataFrame([(rank, drug.split('_')[1]) for rank,drug in enumerate(le.inverse_transform((types=='drug').nonzero()[0][top_items_idx])[:topk+1])], columns=['rank', 'drug'])
topk_drugs['under_trials']=topk_drugs['drug'].isin(trials_drug).astype(int)
topk_drugs['is_used_in_training']=topk_drugs['drug'].isin(np.array([drug.split('_')[1] for drug in le.classes_[types=='drug']])[indices_train]).astype(int)
topk_drugs.to_csv('top300_drugs_v2.csv')